<a href="https://colab.research.google.com/github/SuperRAS/SuperRAS/blob/main/Exercicio_D9_10_1_2_Romualdo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ETL dos dados referentes à participação do agronegócio no PIB nacional e dos desembolsos feitos pelo sistema BNDES ao setor.

###O objetivo da tarefa é verificar a relação entre o desempenho do agronegócio ao longo dos anos e identificar eventual correlação entre o fomento do setor pelo BNDES e a participação do agronegócio na economia do país.

###Os dados do PIB e da participação do agronegócio foram extraídos do Centro de Estudos Avançados em Economia Aplicada, da Escola Superior de Agricultura Luiz de Queiroz, da Universidade de São Paulo - CEPEA/ESALQ/USP: https://www.cepea.esalq.usp.br/upload/kceditor/files/Planilha_PIB_Cepea_Portugues_Site_2020.xlsx

###Para os dados sobre desembolsos do sistema BNDES, a fonte foi o portal de dados abertos do governo federal no seguinte link: https://api.bcb.gov.br/dados/serie/bcdata.sgs.7418/dados?formato=csv


In [35]:
import pandas as pd
from datetime import date
import numpy as np
project_id = 'my-project-256389-331520' 




###1)Extração e tratamento dos dados relacionados ao PIB do agronegócio e construção de dataframe.

In [2]:
df_pibagr = pd.read_excel('https://github.com/SuperRAS/SuperRAS/raw/main/Planilha_PIB_Cepea_Portugues_Site_2020.xls')
from datetime import date
pd.to_datetime(df_pibagr['data'], format='%Y')
df_pibagr['ano'] = df_pibagr['data'].dt.year
df_pibagr.head()

,data,pib_agro,pib_brasil,agro_pib_brasil,ano
0,1996-01-01,297653,854763,34.822869,1996
1,1997-01-01,298507,952089,31.352846,1997
2,1998-01-01,298722,1002351,29.802135,1998
3,1999-01-01,319589,1087710,29.381821,1999
4,2000-01-01,365411,1199092,30.473975,2000


In [27]:
df_pibagr.set_index('ano',inplace=True)

In [28]:
df_pibagr.dtypes

data               datetime64[ns]
pib_agro                    int64
pib_brasil                  int64
agro_pib_brasil           float64
dtype: object

###2)Extração e tratamento dos dados relacionados aos desembolsos do sistema BNDES ao agronegócio e construção de dataframe.
Agrupamento dos dados por ano.

In [29]:
df_bn=pd.read_csv('https://api.bcb.gov.br/dados/serie/bcdata.sgs.7418/dados?formato=csv', encoding='utf8', sep=';')
from datetime import date, datetime
pd.to_datetime(df_bn['data'], format='%d/%m/%Y')
df_bn['data']= pd.to_datetime(df_bn['data'])
df_bn['ano'] = df_bn['data'].dt.year
df_bngrouped = df_bn.groupby(['ano']).sum().reset_index()
df_bngrouped.head(26)


,ano,valor
0,1995,4868
1,1996,4704
2,1997,7830
3,1998,7953
4,1999,8033
5,2000,10664
6,2001,16147
7,2002,24575
8,2003,24824
9,2004,43278


###3)Merge dos dataframes de PIB do agronegócio e dos desembolsos do sistema BNDES.

In [30]:
df_pibn = df_pibagr.merge(df_bngrouped, on=['ano'], how='left')
del df_pibn['data']
df_pibn.head(25)

,ano,pib_agro,pib_brasil,agro_pib_brasil,valor
0,1996,297653,854763,34.822869,4704
1,1997,298507,952089,31.352846,7830
2,1998,298722,1002351,29.802135,7953
3,1999,319589,1087710,29.381821,8033
4,2000,365411,1199092,30.473975,10664
5,2001,395196,1315755,30.035683,16147
6,2002,448823,1488787,30.146891,24575
7,2003,523607,1717950,30.478594,24824
8,2004,536436,1957751,27.400624,43278
9,2005,529142,2170584,24.377863,27221


###4)Ajustes nos nomes das colunas para melhor identificação.

In [31]:
df_agrpibn = df_pibn.rename(columns={'ano': 'ano', 'pib_agro': 'pib_agro', 'pib_brasil':'pib_brasil', 'agro_pib_brasil':'agro_brasil', 'valor':'des_bndes'})

###5)Ordenação lógica das colunas e arredondamento para duas casas decimais da coluna participação do agronegócio no PIB nacional.

In [33]:
df_agrpibn[['ano', 'pib_agro', 'pib_brasil', 'agro_brasil', 'des_bndes']]
df_agrpibn.round(2)


,ano,pib_agro,pib_brasil,agro_brasil,des_bndes
0,1996,297653,854763,34.82,4704
1,1997,298507,952089,31.35,7830
2,1998,298722,1002351,29.80,7953
3,1999,319589,1087710,29.38,8033
4,2000,365411,1199092,30.47,10664
5,2001,395196,1315755,30.04,16147
6,2002,448823,1488787,30.15,24575
7,2003,523607,1717950,30.48,24824
8,2004,536436,1957751,27.40,43278
9,2005,529142,2170584,24.38,27221


In [37]:
df_agrpibn.to_gbq("enapdatasets.agroparticipa",
  project_id=project_id,
  chunksize=40000,
  if_exists='replace',
  )

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=725825577420-unm2gnkiprugilg743tkbig250f4sfsj.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=xD6BM71P1TEbkqbmbU8S4KNpbQgdrZ&prompt=consent&access_type=offline
Enter the authorization code: 4/1AX4XfWhMVuvddeKihpDC1vwax4gm89kN9vebrwdy3-U8ZCS821NX7amULOM


1it [00:03,  3.14s/it]
